In [24]:
import json
with open('/opt/ml/level3_nlp_finalproject-nlp-11/T5/test_questions.json','r') as f:
    input_text = json.load(f)

In [25]:
input_text

['who got the first nobel prize in physics',
 'when is the next deadpool movie being released',
 'which mode is used for short wave broadcast service',
 'the south west wind blows across nigeria between',
 'what does hp mean in war and order',
 'who wrote the first declaration of human rights',
 'who is the owner of reading football club',
 'when is the next scandal episode coming out',
 'when is the last time the philadelphia won the superbowl',
 'who was the first lady nominated member of the rajya sabha',
 'what is the most current adobe flash player version',
 'swan lake the sleeping beauty and the nutcracker are three famous ballets by',
 'how many episodes are there in dragon ball z',
 'cast of law & order special victim unit',
 'who designed the garden city of new earswick',
 'what is the first step in the evolution of the eye',
 'where is the tv show the curse of oak island filmed',
 'where is gall bladder situated in human body',
 'who is the current director of the us mint',


In [27]:
from densephrases.utils.squad_utils import TrueCaser
path = "/opt/ml/level3_nlp_finalproject-nlp-11/DensePhrases/densephrases-data/truecase/english_with_questions.dist"
truecase = TrueCaser(path)
changed_input = []
for input in input_text:
    changed_input.append(truecase.get_true_case(input))

07/26/2023 05:35:35 - INFO - densephrases.utils.file_utils -   TensorFlow version 2.13.0 available.


In [29]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch
import tqdm
model_name = "/opt/ml/level3_nlp_finalproject-nlp-11/T5/t5_reparaphrase" #허깅
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
max_token_length = 64
train_data = torch.utils.data.DataLoader(changed_input, batch_size=256)
results=[]
for i_bacth,td in enumerate(tqdm.tqdm(train_data)):
    inputs = tokenizer(td, return_tensors="pt", padding=True).to(device)
    output_sequences = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    result=tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
    results.append(result)

07/26/2023 05:36:46 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model from cache at /opt/ml/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f
07/26/2023 05:36:46 - INFO - transformers.configuration_utils -   loading configuration file /opt/ml/level3_nlp_finalproject-nlp-11/T5/t5_reparaphrase/config.json
07/26/2023 05:36:46 - INFO - transformers.configuration_utils -   Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_nu

TypeError: 'T5Tokenizer' object is not callable

In [14]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('/opt/ml/level3_nlp_finalproject-nlp-11/T5/t5_paraphrase')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

sentence = "the south west wind blows across nigeria between?"
# sentence = "What are the ingredients required to bake a perfect cake?"
# sentence = "What is the best possible approach to learn aeronautical engineering?"
# sentence = "Do apples taste better than oranges in general?"


text =  "paraphrase: " + sentence + " </s>"


max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    top_k=120,
    top_p=0.98
)


print ("\nOriginal Question ::")
print (sentence)
print ("\n")
print ("Paraphrased Questions :: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

device  cuda

Original Question ::
the south west wind blows across nigeria between?


Paraphrased Questions :: 
0: What is the name of the wind that blows across Nigeria between the South West and the West?


In [30]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import DataLoader

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('/opt/ml/level3_nlp_finalproject-nlp-11/T5/t5_paraphrase')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device ", device)
model = model.to(device)

sentences = changed_input

batch_size = 256
max_len = 256

def preprocess(sentences):
    inputs = []
    for sentence in sentences:
        text = "paraphrase: " + sentence + " </s>"
        encoding = tokenizer.encode_plus(text, pad_to_max_length=True, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
        inputs.append((input_ids, attention_masks))
    return inputs

def generate_paraphrases(inputs):
    input_ids, attention_masks = zip(*inputs)
    input_ids = torch.cat(input_ids)
    attention_masks = torch.cat(attention_masks)

    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        max_length=256,
        top_k=120,
        top_p=0.98
    )

    final_outputs = []
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        final_outputs.append(sent)

    return final_outputs

# Split the sentences into batches
batches = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]

paraphrases = []
for batch in batches:
    inputs = preprocess(batch)
    outputs = generate_paraphrases(inputs)
    paraphrases.extend(outputs)

# print("\nOriginal Questions:")
# for sentence in sentences:
#     print(sentence)

# print("\nParaphrased Questions:")
# for i, paraphrase in enumerate(paraphrases):
#     print("{}: {}".format(i, paraphrase))

07/26/2023 05:37:21 - INFO - transformers.configuration_utils -   loading configuration file /opt/ml/level3_nlp_finalproject-nlp-11/T5/t5_paraphrase/config.json
07/26/2023 05:37:21 - INFO - transformers.configuration_utils -   Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,

device  cuda


In [31]:
len(paraphrases)

3610

In [4]:
sum(map(len,results))

3610

In [5]:
r=[]
for result in results:
    r+=result

In [6]:
r

['Who received the first Nobel Prize in Physics?',
 'When will the next Deadpool movie be released?',
 'Which mode is used for short wave broadcasting?',
 'Between which regions of Nigeria does the South West Wind blow?',
 'What does HP stand for in the TV show "War and Order"?',
 'Who wrote the first Declaration of Human Rights?',
 'Who is the owner of Reading Football Club?',
 'When will the next episode of "Scandal" be released?',
 'When was the last time the Philadelphia Eagles won the Super Bowl?',
 'Who was the first female member nominated to the Rajya Sabha?',
 'What is the most recent version of Adobe Flash Player?',
 'What are three famous ballets by Swan Lake, Sleeping Beauty, and The Nutcra',
 'How many episodes are there in Dragon Ball Z?',
 'Who are the cast members of the Law & Order Special Victim Unit?',
 'Who designed the Garden City of New Earswick?',
 'What is the first step in the evolution of the eye?',
 'Where is the TV show "The Curse of Oak Island" filmed?',
 '

In [11]:
r[30]

'Which country has a panda as its national animal?'

In [32]:
with open('test_preprocessed_changed_with_True_t5.txt','w',encoding='utf-8') as f:
    for i in paraphrases:
        f.write(i+'\n')